<a href="https://colab.research.google.com/github/westefns-souza/Trabalho_analise_redes_twitter/blob/main/Trabalho_analise_redes_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analise da base de dados do twitter



```
https://www.kaggle.com/datasets/hwassner/TwitterFriends
```



In [ ]:
!pip install pyvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.2 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
import networkx as nx
from pyvis.network import Network
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import csv

# Limpando a base
Como a base não tinha um csv bom, tive que limpar principalmente a coluna 'friends' que estava como um array separado por vírgula, com isso a leitura quebrava no momento de abrir e ler os dados com o pandas.

Achei Uma função de substituir_virgulas usando regex para identicar apenas as frases dentro de um array [ ].

In [ ]:
def substituir_virgulas(texto):
    padrao = r'\[(.*?)\]'  # padrão para encontrar o texto entre colchetes
    matches = re.findall(padrao, texto)  # encontrar todas as ocorrências entre colchetes
    for match in matches:
        novo_texto = match.replace(',', ';')  # substituir vírgulas por ponto-e-vírgula
        texto = texto.replace(match, novo_texto)  # substituir a ocorrência original pelo novo texto
    return texto

## Criando um novo arquivo já corrigido

Para que não precise trabalhar com os dados em memória sempre

In [ ]:
with open("/content/data_twitter.csv") as file:
  with open("/content/data_twitter_formated.csv", "w") as new_file:
      for line in file:
        new_file.write(substituir_virgulas(line))

FileNotFoundError: ignored

Carregando os dados do arquivo formatado para um DateFrame

In [ ]:
df = pd.read_csv('/content/data_twitter_formated.csv', on_bad_lines="skip")
df

Selecionando apenas as linhas que estou utilizando

In [ ]:
df_reduced = df[['id', 'friends']]
df_reduced

Separando as edges juntamente com os nós, foi necessário porque o coluna de friends se torna uma lista

In [ ]:
nodes = []
edges = []

In [ ]:
for line in df_reduced.values.tolist():
  userId = int(line[0])
  nodes.append(userId)

  friends = line[1].replace('[', '').replace('; ]', '').replace(']', '').replace('"', '').split(';')

  for friend in friends:
    friend = friend.strip()
    if friend != '':
      friend = int(friend)
      nodes.append(friend)
      edges.append([userId, friend])

Criando um arquvivo apenas com os nós para usar depois sem precisar dos passos acima

In [ ]:
with open('base_reduzida.csv', 'w', newline='') as csvfile:
    fieldnames = ['userId', 'friendId']
    writer = csv.DictWriter(csvfile, fieldnames)

    writer.writeheader()
    for linha in edges:
      writer.writerow({'userId': linha[0], 'friendId': linha[1]})

# Usando apartir da base reduzida para analizar os dados

In [ ]:
df_base_reduzida = pd.read_csv('/content/base_reduzida.csv', on_bad_lines="skip")

In [ ]:
import random

def reduzir_base_amostragem_aleatoria(dados, tamanho_amostra):
    amostra = random.sample(dados, tamanho_amostra)
    return amostra

# Exemplo de uso
amostra_reduzida = reduzir_base_amostragem_aleatoria(df_base_reduzida.values.tolist(), 50000)
amostra_reduzida

In [ ]:
len(amostra_reduzida)

## Criando o Grafico com NetworkX

In [ ]:
G = nx.Graph()

for line in df_base_reduzida.values.tolist():
#for line in amostra_reduzida:
  G.add_nodes_from(line)
  G.add_edges_from([(line[0], line[1])])

In [ ]:
len(G.nodes)

In [ ]:
len(G.edges)

In [ ]:
#filter nodes with degree greater than 2
#core decompo = 3
core = [node for node, drg in dict(G.degree()).items() if drg > 1000 ]
#degrees = [drg for node, drg in dict(G.degree()).items() if drg > 700 ]

In [ ]:
subgraf_g = nx.subgraph(G,core)

In [ ]:
print(f'Nodes: {len(subgraf_g)} edges: {nx.number_of_edges(subgraf_g)}')

In [ ]:
nx.write_graphml(G, "network_analysis.graphml")

In [ ]:
nx.write_graphml(subgraf_g, "network_analysis_subgraf.graphml")

## Gerando o gráfico com o Pyvis

In [ ]:
nt = Network('500px', '100%', notebook=True, cdn_resources='in_line')
nt.from_nx(G)
nt.show('grafico.html')
display(HTML('grafico.html'))

# Obtendo os dados pedidos na máteria

## Obtendo a matriz de adjacência

In [ ]:
adj_matrix = nx.to_pandas_adjacency(G)

plt.imshow(adj_matrix, cmap='binary', origin='upper')
plt.xlabel('Nós')
plt.ylabel('Nós')
plt.title('Matriz de Adjacência')
plt.colorbar()
plt.show()

## Calcular o diâmetro

In [ ]:
diameter = nx.diameter(G)
print("Diâmetro: ", diameter)

## Calcular a periferia

In [ ]:
periphery = nx.periphery(G)

print("Periferia: ", periphery)

 ## Histograma de distribuição empírica de grau.

In [ ]:
hist = nx.degree_histogram(G)
degrees = list(range(len(hist)))
frequencies = hist

nt = Network(height="500px", width="100%", notebook=True,cdn_resources='in_line')
nt.from_nx(G)

for degree, freq in zip(degrees, frequencies):
    nt.add_node(degree, size=freq)

nt.show("histograma.html")
display(HTML('histograma.html'))

## Coeficiente de clustering local para nós escolhidos.



In [ ]:
coefs_clustering = nx.clustering(G, [1969527638])
coefs_clustering

## Calcular o coeficiente de clustering global

In [ ]:
coef_clustering = nx.average_clustering(G)
coef_clustering

## Encontrar os componentes conectados fortemente

In [ ]:
componentes_fortemente = nx.strongly_connected_components(G)
componentes_fortemente

## Encontrar os componentes conectados fracamente

In [ ]:
componentes_fracamente = nx.weakly_connected_components(G)
componentes_fracamente

## Calcular a centralidade de grau

In [ ]:
centrality = nx.degree_centrality(G)

for node, centrality_value in centrality.items():
    print("Nó:", node, "Centralidade de Grau: ", centrality_value)

## Calcular a centralidade de proximidade

In [ ]:
centrality = nx.closeness_centrality(G)

for node, centrality_value in centrality.items():
    print("Nó:", node, "Centralidade de Proximidade: ", centrality_value)

## Calcular o Betweenness centrality

In [ ]:
centrality = nx.betweenness_centrality(G)

for node, centrality_value in centrality.items():
    print("Nó:", node, "Centralidade de Intermediação: ", centrality_value)

## Calcular a centralidade de eigenvector

In [ ]:
centrality = nx.eigenvector_centrality(G)

for node, centrality_value in centrality.items():
    print("Nó:", node, "Centralidade de Eigenvector: ", centrality_value)

## Calcular a assortatividade geral

In [ ]:
assortativity = nx.degree_assortativity_coefficient(G)
assortativity